In [1]:
pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 39.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
import numpy as np

In [ ]:
import pandas as pd

# 파일 경로 설정
parquet_path = "yellow_tripdata_2024-08.parquet"
csv_output_path = "diifs2.csv"

# Parquet 파일 읽기
df = pd.read_parquet(parquet_path)

# 픽업 및 드롭오프 시간 추출
df['pickup_datetime'] = pd.to_datetime(df.iloc[:, 1])
df['dropoff_datetime'] = pd.to_datetime(df.iloc[:, 2])

# 기준점: 2024년 1월 1일 00:00:00
reference_time = pd.Timestamp("2024-06-30 00:00:00")

# 기준점부터의 초 단위 차이 계산
pickup_seconds = (df['pickup_datetime'] - reference_time).dt.total_seconds().astype(int)
dropoff_seconds = (df['dropoff_datetime'] - reference_time).dt.total_seconds().astype(int)

# 새로운 DataFrame 생성
normalized_df = pd.DataFrame({
    'pickup_seconds': pickup_seconds,
    'dropoff_seconds': dropoff_seconds
})

# CSV 파일로 저장
normalized_df.to_csv(csv_output_path, index=False)

print(f"Normalized intervals saved to: {csv_output_path}")


In [ ]:
import pandas as pd

# 입력 CSV 파일과 출력 CSV 파일 경로 설정
input_csv_path = "yellow_tripdata_timestamp_diffs.csv"
output_csv_path = "TAXIS.csv"

# CSV 파일 읽기
df = pd.read_csv(input_csv_path)

# 'pickup_seconds'와 'dropoff_seconds' 컬럼을 가져오기
start_points = df['pickup_seconds']
end_points = df['dropoff_seconds']

# start_point의 최솟값 계산
min_start_point = start_points.min()

print(min_start_point)

# 모든 start_point와 end_point에서 최솟값을 빼기
df['start_point'] = start_points - min_start_point
df['end_point'] = end_points - min_start_point

# 변환된 결과로 새로운 DataFrame 생성
shifted_df = df[['start_point', 'end_point']]

# CSV 파일로 저장
# shifted_df.to_csv(output_csv_path, index=False)

# print(f"Shifted intervals saved to: {output_csv_path}")


In [ ]:
import pandas as pd

# CSV 파일 경로 설정
input_csv_path = "TAXIS.csv"
output_csv_path = "cleaned_TAXIS.csv"

# CSV 파일 읽기 (첫 번째 열이 start_point, 두 번째 열이 end_point)
df = pd.read_csv(input_csv_path, header=None, names=["start_point", "end_point"])

# 1. start_point > end_point 인 경우 제거
df = df[df['start_point'] <= df['end_point']]

# 2. 음수 값은 이번 데이터에서는 없으므로 처리할 필요가 없습니다.

# 3. 길이 (end_point - start_point)가 0인 경우 제거
df = df[df['end_point'] - df['start_point'] > 0]

# 정제된 데이터 저장
df.to_csv(output_csv_path, index=False)

print(f"Cleaned data saved to: {output_csv_path}")


In [ ]:
import pandas as pd

# 파일 로드
file_path = "BOOKS.csv"
df = pd.read_csv(file_path, header=None, names=["start_point", "end_point"])

# 메트릭 계산
cardinality = len(df)  # Cardinality: 행의 개수
domain_size = df["end_point"].max() - df["start_point"].min()  # Domain size
min_length = (df["end_point"] - df["start_point"]).min()  # Min length
max_length = (df["end_point"] - df["start_point"]).max()  # Max length
avg_length = (df["end_point"] - df["start_point"]).mean()  # Avg length

# 결과 출력
print("Cardinality:\n", cardinality)
print("\nDomain size:\n", domain_size)
print("\nMin length:\n", min_length)
print("\nMax length:\n", max_length)
print("\nAvg length:\n", avg_length)
print("percentage", avg_length / domain_size * 100)

Cardinality:
 6053995

Domain size:
 6208602

Min length:
 1

Max length:
 580217

Avg length:
 1037.9356692564165
percentage 0.016717703425930937
